# Generate a DomoStats style Dataset

Generating the Dataset is just a case of getting your data into the shape that you want it.

Developer Tip
- separate the act of retrieving data from act of restructuring your data for use

In [17]:
import domojupyter as domo
import json

def get_account_credentials(account_name, is_abstract_account: bool = True):
    """handles retrieving account properties in domo jupter"""
    
    account_properties = domo.get_account_property_keys(account_name)
    res = {prop: domo.get_account_property_value(account_name, prop) for prop in account_properties}
    return json.loads(res['credentials'])

creds = get_account_credentials("username_password_auth", is_abstract_account=True)
domo_instance = creds['DOMO_INSTANCE']

In [8]:
class DomoAPIRequest_Error(Exception):
    """a customized Exception class for handing Domo errors"""
    
    def __init__(self, message):
        super().__init__(message)

# Domo Authentication

In [9]:
import requests

def get_full_auth(
    domo_instance: str,
    domo_username: str,
    domo_password: str
) -> str:  # returns a session token
    """use username and password to generate an access token"""

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    res = requests.request(method="POST", url=url, json=body)
    data = res.json()

    token = data.get("sessionToken")
    
    if not token:
        raise DomoAPIRequest_Error("unable to retrieve a session token")

    return token

session_token = get_full_auth(domo_username = creds['DOMO_USERNAME'],
                              domo_password = creds['DOMO_PASSWORD'],
                              domo_instance = creds['DOMO_INSTANCE'] )
assert session_token

In [10]:
import json

# to accommod
def get_account_credentials(account_name, is_abstract_account: bool = True):
    
    account_properties = domo.get_account_property_keys(account_name)
    res = {prop: domo.get_account_property_value(account_name, prop) for prop in account_properties}
    return json.loads(res['credentials'])

creds = get_account_credentials("username_password_auth")
# creds

# Interact with Domo

In [25]:
from typing import List

def get_accounts(domo_instance, session_token) -> List[dict]:
    """retrieves a list of accounts this user has access to"""
    
    headers = {"x-domo-authentication": session_token}    
    url = f'https://{domo_instance}.domo.com/api/data/v2/datasources/providers'
    
    res= requests.request(method = "GET",
                            url = url,
                            headers = headers
                           )
    
    data = res.json()
    
    if not res.ok:
        DomoAPIRequestError(data)
        
    return data

# How should we begin?
Look at what the API, `get_accounts` gives you -- a `List[dict]` where each dict represents a `dataProiderType` and within each `dataProviderType` there's the list of `accounts` that belong to that data provider type.

Express your Target granularity
"We want one row for each account"

DEVELOPER_NOTE
- while it is common for APIs to return JSON in camelCase, in our code we will rewrite everything in snake_case.
- we can also take the liberty of renaming properties to something user friendly

## utils

In [34]:
import re

def format_str_camel_case(text):
    # https://www.w3resource.com/python-exercises/string/python-data-type-string-exercise-97.php
    # Replace hyphens with spaces, then apply regular expression substitutions for title case conversion
    # and add an underscore between words, finally convert the result to lowercase

    return '_'.join(
        re.sub('([A-Z][a-z]+)', r' \1',
        re.sub('([A-Z]+)', r' \1',
        text.replace('-', ' '))).split()).lower() 

print(format_str_camel_case("doesThisWork?"))
print(format_str_camel_case("what about This?"))

does_this_work?
what_about_this?


## process API Response

In [78]:
test_data = get_accounts(domo_instance = domo_instance, session_token = session_token)
test_data_provider_type = test_data[0]
test_data_provider_type

{'key': 'abstract-credential-store',
 'name': 'Abstract Credential Store',
 'authenticationScheme': 'fields',
 'unassociatedDataSourceCount': 0,
 'accounts': [{'id': 71,
   'name': 'domo_creds',
   'userId': '1893952720',
   'displayName': 'domo_creds - update 2024-02-21',
   'type': 'data',
   'dataProviderType': 'abstract-credential-store',
   'valid': True,
   'dateOfExpiration': None,
   'dataSourceCount': 0,
   'daysToExpiry': None,
   'expired': None},
  {'id': 87,
   'name': 'Abstract Credential Store Account',
   'userId': '1893952720',
   'displayName': 'jw_creds',
   'type': 'data',
   'dataProviderType': 'abstract-credential-store',
   'valid': True,
   'dateOfExpiration': None,
   'dataSourceCount': 0,
   'daysToExpiry': None,
   'expired': None},
  {'id': 88,
   'name': 'fake_account',
   'userId': '1893952720',
   'displayName': 'fake_account',
   'type': 'data',
   'dataProviderType': 'abstract-credential-store',
   'valid': True,
   'dateOfExpiration': None,
   'dataSou

In [68]:
test_data_provider_type['accounts'][0] 

{'name': 'domo_creds',
 'userId': '1893952720',
 'displayName': 'domo_creds - update 2024-02-21',
 'type': 'data',
 'dataProviderType': 'abstract-credential-store',
 'valid': True,
 'dateOfExpiration': None,
 'dataSourceCount': 0,
 'daysToExpiry': None,
 'expired': None,
 'data_provider_type_display_name': 'Abstract Credential Store',
 'data_provider_type_name': 'Abstract Credential Store'}

In [94]:
def process_domostats_get_accounts_account(account_obj, data_provider_type_name):
    """most granular level"""
    
    account_obj.update({"data_provider_name": data_provider_type_name}) 
    
    account_obj['dataProviderType'] = account_obj['dataProviderType'] 
    account_obj['account_id'] = account_obj.pop('id')
    account_obj['account_name'] = account_obj.pop('displayName')

    account_obj.pop('name')
    account_obj.pop('type')
    account_obj.pop('daysToExpiry')
    account_obj.pop('valid')
    account_obj.pop('expired')

    
    return { format_str_camel_case(key) : value for key, value in account_obj.items()}

    
def process_domostats_get_accounts_data_povider(data_provider_obj) -> List[dict]:
    """receives the data_provider obj and flattens to the account obj"""
    account_ls = data_provider_obj['accounts']
    
    return [process_domostats_get_accounts_account(account_obj.copy(), data_provider_type_name = data_provider_obj['name']) for account_obj in account_ls]
    
process_domostats_get_accounts_dataprovider(test_data_provider_type.copy())[0:5]

[{'user_id': '1893952720',
  'data_provider_type': 'abstract-credential-store',
  'date_of_expiration': None,
  'data_source_count': 0,
  'data_provider_name': 'Abstract Credential Store',
  'account_id': 71,
  'account_name': 'domo_creds - update 2024-02-21'},
 {'user_id': '1893952720',
  'data_provider_type': 'abstract-credential-store',
  'date_of_expiration': None,
  'data_source_count': 0,
  'data_provider_name': 'Abstract Credential Store',
  'account_id': 87,
  'account_name': 'jw_creds'},
 {'user_id': '1893952720',
  'data_provider_type': 'abstract-credential-store',
  'date_of_expiration': None,
  'data_source_count': 0,
  'data_provider_name': 'Abstract Credential Store',
  'account_id': 88,
  'account_name': 'fake_account'},
 {'user_id': '1893952720',
  'data_provider_type': 'abstract-credential-store',
  'date_of_expiration': None,
  'data_source_count': 0,
  'data_provider_name': 'Abstract Credential Store',
  'account_id': 92,
  'account_name': 'jw_username_password_auth'

In [99]:
import pandas as pd

def generate_domostats_get_accounts(
    domo_instance, session_token
) -> pd.DataFrame:
    
    api_response = get_accounts(domo_instance = domo_instance, session_token = session_token)
    
    accounts_by_providers_ls = [ process_domostats_get_accounts_data_povider(data_provider_obj) for data_provider_obj in api_response]
    
    # return accounts_by_providers_ls # one list of accounts per data_provider_type
    
    account_ls = [account for account_ls in accounts_by_providers_ls for account in account_ls] 
    
    # return account_ls # a list of accounts
    
    return pd.DataFrame(account_ls)

generate_domostats_get_accounts(
    domo_instance = domo_instance,
    session_token = session_token
)[0:5]
    

,user_id,data_provider_type,date_of_expiration,data_source_count,data_provider_name,account_id,account_name
0,1893952720,abstract-credential-store,None,0,Abstract Credential Store,71,domo_creds - update 2024-02-21
1,1893952720,abstract-credential-store,None,0,Abstract Credential Store,87,jw_creds
2,1893952720,abstract-credential-store,None,0,Abstract Credential Store,88,fake_account
3,1893952720,abstract-credential-store,None,0,Abstract Credential Store,92,jw_username_password_auth
4,1893952720,dataset-copy,None,1,DataSet Copy,1,dsa - northshore


## 🚧 HOMEWORK!!

Notice that the named user_id is just a user_id is just a name.

can you construct a function, `get_user_by_id` that GETs user information from this api?

url = f'https://{domo_instance}.domo.com/api/content/v2/users/{user_id}

Then can you alter `process_domostats_get_accounts_account` to call the `get_user_by_id` function for the `userId` (remember it gets renamed to `user_id` after the application of the `format_str_camel_case` function of the ) and return the user_name?

## Output as a Dataframe

To output as a dataframe, modify the configuration of the JupyterWorkspace to have an output file.

1. add an output dataset to domojupyter workspace to interact with<br><br>
   Data > Jupyter Workspaces > Edit (Workspace Name) > Output Datasets > Add Output Dataset - "DomoStas - Accounts"

2. call `domojupyter.write_dataframe` to output the dataset.  Note it is theoretically possible to apply a PARTITION or UPSERT scheme instead of a straight REPLACE

In [101]:
import domojupyter as domo

def main():
    
    creds = get_account_credentials('username_password_auth')
    domo_instance = creds['DOMO_INSTANCE']
    
    session_token = get_full_auth(domo_username = creds['DOMO_USERNAME'],
                                  domo_password = creds['DOMO_PASSWORD'],
                                  domo_instance = domo_instance
                                )

    df = generate_domostats_get_accounts(domo_instance = domo_instance,
                                         session_token = session_token)
    

    domo.write_dataframe(df, 'DomoStats - Accounts')
    
    
main()

/home/domo/.conda/lib/python3.9/site-packages/domojupyter/io.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  columns = [{'name': column, 'type': _convert_type(df.dtypes[ind].name)} for ind, column in enumerate(df.columns)]
/home/domo/.conda/lib/python3.9/site-packages/domojupyter/io.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  columns = [{'name': column, 'type': _convert_type(df.dtypes[ind].name)} for ind, column in enumerate(df.columns)]
/home/domo/.conda/lib/python3.9/site-packages/domojupyter/io.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, 